In [21]:
import pandas as pd
from scipy.spatial.distance import euclidean
from process import *
from utils import *
from model import *

In [22]:
#prendo i dati relativi al dataset
device='cpu'
layer=3
adj, adj_i, features, labels, idx_train, idx_val, idx_test, num_features, num_labels, g = full_load_data_custom('citeseer','splits/citeseer_split_0.6_0.2_0.npz')
features = features.to(device)

# processo la matrice di adiacenza del grafo per darlo in input alla rete
adj = adj.to(device)
adj_i = adj_i.to(device)
list_mat = []
list_mat.append(features)
no_loop_mat = features
loop_mat = features

for ii in range(layer):
    no_loop_mat = torch.spmm(adj, no_loop_mat)
    loop_mat = torch.spmm(adj_i, loop_mat)
    list_mat.append(no_loop_mat)
    list_mat.append(loop_mat)

In [23]:
layer_norm=bool(1)
# Define the path to the saved model checkpoint
PATH = "citeseer\citeseer.pt"
GNNciteseer=FSGNN(nfeat=num_features,
                nlayers=len(list_mat),
                nhidden=64,
                nclass=num_labels,
                dropout=0.6).to(device)
GNNciteseer.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [24]:
#prendo l'embedding dalla rete
GNNciteseer.eval()
out = GNNciteseer.emb(list_mat, layer_norm)
#torch.save(out,'citeseer\emb_citeseer25.pt')


In [25]:
out.shape

torch.Size([3327, 448])

In [26]:
#trasformo la matrice di adiacenza in grafo
G = nx.Graph(g)
adj_mat=pd.DataFrame(columns=['target','source'])
adj_mat = nx.to_pandas_edgelist(G)
adj_mat = adj_mat[adj_mat['source']!=adj_mat['target']]
adj_mat

,source,target,weight
0,0,628,1
1,1,158,1
2,1,486,1
3,1,1097,1
4,1,2919,1
...,...,...,...
4670,3256,3281,1
4672,3286,3287,1
4673,3301,3302,1
4674,3319,3320,1


In [27]:
#definizione distanza manhattan
import numpy as np

def manhattan_distance(arr1, arr2):
    """
    Calculates the Manhattan distance between two arrays of the same length.
    """
    return np.sum(np.abs(arr1 - arr2))

In [28]:
'''#aggiungo i pesi ottenuti dall'embedding
df_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])
adj_mat['weight'] = adj_mat.apply(lambda x:
                                    manhattan_distance(df_emb.loc[x['target']].values,
                                    df_emb.loc[x['source']].values),
                                    axis=1)
adj_mat.to_csv('citeseer\citeseer_weights.csv', index = False)'''

"#aggiungo i pesi ottenuti dall'embedding\ndf_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])\nadj_mat['weight'] = adj_mat.apply(lambda x:\n                                    manhattan_distance(df_emb.loc[x['target']].values,\n                                    df_emb.loc[x['source']].values),\n                                    axis=1)\nadj_mat.to_csv('citeseer\\citeseer_weights.csv', index = False)"

In [29]:
#aggiungo i pesi ottenuti dall'embedding
from sklearn.metrics.pairwise import cosine_similarity

df_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])
adj_mat['weight'] = adj_mat.apply(lambda x:
                                    cosine_similarity(np.array([df_emb.iloc[x['target']]]),
                                    np.array([df_emb.iloc[x['source']]]))[0][0],
                                    axis=1)
adj_mat.to_csv('citeseer/citeseer_cosine_weights.csv', index = False)

In [ ]:
#label della rete 
pd.DataFrame(labels.numpy(),columns=['label']).to_csv('citeseer\citeseer_labels.csv', index = False)